In [ ]:
# Install and import the necessary librarie
!pip install torch
!pip install -q -U accelerate peft bitsandbytes transformers einops
!pip install language_tool_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 8.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.9 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 96.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 10.9 MB/s eta 0:00:00a 0:00:01


In [3]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [148]:
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

In [4]:
test_df=pd.read_csv("/kaggle/input/juridia-hackhaton-fine-tuning-llm-v/test.csv")

In [6]:
# Model
#base_model = "microsoft/Phi-3-small-8k-instruct" #"mistralai/Mistral-Nemo-Base-2407"
base_model= "microsoft/Phi-3.5-mini-instruct"

# Dataset
#dataset = load_dataset("prsdm/MedQuad-phi2-1k", split="train")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True,trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="left"

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [7]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    device_map={"": 0},
    #revision="refs/pr/23" #the main version of Phi-2 doesn’t support gradient checkpointing (while training this model)
)

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [10]:
from peft import PeftModel

# Load LoRA model
peft_model = PeftModel.from_pretrained(model, "/kaggle/input/juridia-model/results/checkpoint-1500")


In [136]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

sum_tokenizer = AutoTokenizer.from_pretrained("plguillou/t5-base-fr-sum-cnndm")
sum_model = AutoModelForSeq2SeqLM.from_pretrained("plguillou/t5-base-fr-sum-cnndm")

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [149]:
def generate_summary(text: str, model=sum_model, tokenizer=sum_tokenizer, device="cuda") -> str:
    """
    Generate summary using the transformer model with improved settings
    """
    # Move the model to the specified device (e.g., 'cuda' or 'cpu')
    model = model.to(device)
    
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    
    # Move the inputs to the device
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    # Generate the summary
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=256,  # Increased max length for more complete summary
        min_length=120,  # Ensure minimum summary length
        num_beams=4,     # Improved beam search
        length_penalty=0.1,  # Slightly favor shorter sequences
        no_repeat_ngram_size=3,
        early_stopping=True,
        do_sample=False,     # Deterministic output
        temperature=1.0,     # Conservative sampling
        top_k=50,           # Limit vocabulary choices
        repetition_penalty=1.2  # Avoid repetitive phrases
    )

    # Move the output back to the CPU before decoding
    output = output.to('cpu')
    
    # Decode the output and return the summary
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import language_tool_python
import requests
import json
import time

In [12]:
def read_gdrive_json(file_id: str):
    """
    Read JSON from Google Drive file.
    """
    download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    try:
        response = requests.get(download_url)
        response.raise_for_status()
        data = json.loads(response.text)
        return data
    except Exception as e:
        print(f"Error: {e}")
        return []

# Load legal keywords
file_id = "1a9R53Wr8X1oOKDN0iCBT9Yq2A7NETUFE"
legal_keywords = read_gdrive_json(file_id)
legal_pattern = re.compile(r"\b(?:%s)\b" % "|".join(map(re.escape, legal_keywords)), re.IGNORECASE)

# Cached LanguageTool instance
tool = language_tool_python.LanguageTool('fr')  # Adjust to 'ar' for Arabic as needed

def calculate_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates Relevance, Completeness, Legal Soundness, Fluency, and Latency for question-answer pairs.
    """
    # Cached TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Pre-fit TF-IDF for all answers and questions
    all_texts = df["question"].tolist() + df["answer"].tolist()
    tfidf_matrix = vectorizer.fit_transform(all_texts)
    question_matrix = tfidf_matrix[: len(df)]
    answer_matrix = tfidf_matrix[len(df) :]

    # Legal Soundness Check using precompiled regex
    def legal_soundness_check(answer: str) -> float:
        return 1.0 if legal_pattern.search(answer) else 0.0

    # Fluency Check: Batch process to avoid repeated calls
    def fluency_check(answers: list) -> list:
        return [max(0.7, 1 - len(tool.check(answer)) / 10) for answer in answers]

    # Initialize lists for storing metrics
    relevance_scores = cosine_similarity(question_matrix, answer_matrix).diagonal()
    completeness_scores = relevance_scores  # Reuse the similarity scores for completeness
    legal_soundness_scores = [legal_soundness_check(answer) for answer in df["answer"]]
    fluency_scores = fluency_check(df["answer"].tolist())

    # Measure latency
    latency_times = []
    for _, row in df.iterrows():
        start_time = time.time()
        _ = legal_soundness_check(row["answer"])  # Mimic all calculations
        latency_times.append(time.time() - start_time)

    # Add calculated metrics to the DataFrame
    df["relevance"] = relevance_scores
    df["completeness"] = completeness_scores
    df["legal_soundness"] = legal_soundness_scores
    df["fluency"] = fluency_scores
    df["latency"] = latency_times

    return df

def calculate_composite_score(df: pd.DataFrame) -> float:
    """
    Calculate the final weighted composite score based on metrics.
    """
    weights = {
        "relevance": 0.35,
        "completeness": 0.2,
        "legal_soundness": 0.2,
        "fluency": 0.1,
        "latency": -0.05
    }
    weighted_sum = (
        weights["relevance"] * df["relevance"].mean() +
        weights["completeness"] * df["completeness"].mean() +
        weights["legal_soundness"] * df["legal_soundness"].mean() +
        weights["fluency"] * df["fluency"].mean() +
        weights["latency"] * df["latency"].mean()
    )
    return round(weighted_sum, 6)

In [126]:
import torch
from tqdm import tqdm
import pandas as pd

def prompt_template(question):
    return f"""
    ### Instructions:
    Provide a concise, legally sound, complete, and relevant answer to the question below.
    Keep your response brief while ensuring legal validity and clarity.

    ### Question:
    {question}

    ### Answer:
    """

test_df["prompt"] = test_df["question"].apply(prompt_template)


In [164]:


# Model inference function with batching
def generate_answers(df: pd.DataFrame, model, tokenizer, batch_size: int = 1) -> pd.DataFrame:
    """
    Generate answers for the questions in the dataframe using the language model, processing in batches.
    """
    generated_answers = []
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    with torch.no_grad():
        for i in tqdm(range(0, len(df), batch_size)):
            batch_questions = df["prompt"].iloc[i:i + batch_size].tolist()
            inputs = tokenizer(batch_questions, return_tensors="pt", padding=True, truncation=True).to(device)
            input_ids = inputs["input_ids"]
            attention_mask = inputs["attention_mask"]

            output = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                min_length=40,
                max_new_tokens =200,
                num_return_sequences=1,
                temperature=0.75,
                #repetition_penalty=11.0,
                early_stopping=True,
                #do_sample=False,
                #num_beams=2,
                #length_penalty=-10,
                #use_cache=True
            )

            # Remove the input tokens from the output tokens
            batch_generated_texts = []
            for j, generated_seq in enumerate(output):
                input_length = input_ids[j].shape[0]
                generated_text = tokenizer.decode(generated_seq[input_length:], skip_special_tokens=True)
                batch_generated_texts.append(generated_text)

            generated_answers.extend(batch_generated_texts)

    df["answer"] = generated_answers
    return df

In [165]:
# Generate answers for test_df
new_test_df = generate_answers(test_df, peft_model, tokenizer)

  0%|          | 0/220 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.75` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|██████████| 220/220 [55:59<00:00, 15.27s/it]


In [ ]:
new_test_df['answer'] = new_test_df['answer'].progress_apply(lambda x: generate_summary(x))


In [ ]:
new_test_df

,Id,question,answer,relevance,completeness,legal_soundness,fluency,latency,prompt
0,0,Les régimes de protection des personnes fragil...,Les régimes de protection des personnes fragil...,0.009999,0.009999,1.0,0.7,0.000295,\n ### Instructions:\n Provide a concise...
1,1,Je loue un garage. Quelles règles s'appliquent...,1. Le bail doit être écrit et signé par les de...,0.000000,0.000000,0.0,0.7,0.000998,\n ### Instructions:\n Provide a concise...
2,2,Ai-je droit à l'aide juridique (ex pro deo) si...,Être mineur ne donne pas automatiquement droit...,0.000000,0.000000,1.0,0.7,0.000929,\n ### Instructions:\n Provide a concise...
3,3,Peut-on saisir ou confisquer la voiture de mes...,Être impliqué dans le trafic de drogue peut en...,0.007612,0.007612,1.0,0.7,0.000158,\n ### Instructions:\n Provide a concise...
4,4,Quels types de décisions relèvent de l'autorit...,1. Décision sur l'éducation et la formation de...,0.032606,0.032606,1.0,0.7,0.000887,\n ### Instructions:\n Provide a concise...
...,...,...,...,...,...,...,...,...,...
215,215,Je pense que mon logement est insalubre. Je m'...,"1. Rassemblez des preuves de l'insalubrité, te...",0.013408,0.013408,1.0,0.7,0.000349,\n ### Instructions:\n Provide a concise...
216,216,Je suis convoqué en justice. Comment vais-je r...,La décision de justice sera délivrée par le ju...,0.014567,0.014567,1.0,0.7,0.000018,\n ### Instructions:\n Provide a concise...
217,217,Je divorce. Je divorce par consentement mutuel...,Le divorce par consentement mutuel est un proc...,0.008392,0.008392,1.0,0.7,0.000595,\n ### Instructions:\n Provide a concise...
218,218,Je divorce. Je divorce pour cause de désunion ...,1. Désaccord irréductible sur les valeurs fond...,0.000000,0.000000,1.0,0.7,0.000607,\n ### Instructions:\n Provide a concise...


### Experiment on Answer Structuring with Different Templates (Systematic Variation Prompts)

In this experiment, we explore the effect of using different response templates on providing answers to legal questions.

1. **Without Prompt Template**  
   In this case, the answers are provided without any specific template or guidelines. The goal is to see how answers are formulated based on the question.


2. **With Prompt Template (Question-Answer)**  
   This template focuses on formatting the answer in a simple, question-answer structure without applying additional formatting rules or legal constraints. This allows us to observe how the model performs when it's asked to follow a basic template.


3. **With Prompt Template (Legally Sound, Complete, Relevant, and Concise Answers)**  
   In this template, the answers are constrained to be legally valid, fluent, and concise. This version is aimed at producing high-quality, formal responses suitable for legal contexts. We test the model’s ability to filter unnecessary information while maintaining the completeness and legality of the answer.

- English
#Respond only with legally sound, complete, relevant, and concise answers, <br>
#ensuring legal validity, fluency, and completeness.


- French
#Répondez uniquement par des réponses juridiquement solides, complètes,<br>
#pertinentes et concises, en garantissant la validité juridique, la fluidité et l'exhaustivité.

However the french one yield better result


In [152]:
#with prommpt template

#Répondez uniquement par des réponses juridiquement solides, complètes,
#pertinentes et concises, en garantissant la validité juridique, la fluidité et l'exhaustivité.

#this yield the best result
idx=3
new_test_df.iloc[idx]["question"],new_test_df.iloc[idx]["answer"]

('Peut-on saisir ou confisquer la voiture de mes parents que j’utilise pour un trafic de cannabis ?',
 "Le trafic de drogue peut entraîner des conséquences légales, y compris la confiscation de véhicules utilisés dans le crime. La législation spécifique et les procédures varient selon la juridiction. Il est conseillé de consulter un avocat spécialisé en droit pénal pour comprendre les conséquences potentielles à prendre en considération dans le cas d'un impliqué dans le trafic de stupéfiants. Il s'agit notamment de la saisie de voitures.")

In [ ]:
test_df_with_metrics = calculate_metrics(new_test_df)

In [157]:
test_df_with_metrics

,Id,question,answer,relevance,completeness,legal_soundness,fluency,latency
0,0,Les régimes de protection des personnes fragil...,Les régimes de protection des personnes fragil...,0.414866,0.414866,0.0,1.0,0.000826
1,1,Je loue un garage. Quelles règles s'appliquent...,Le bail doit être écrit et signé par les deux ...,0.330525,0.330525,1.0,1.0,0.000021
2,2,Ai-je droit à l'aide juridique (ex pro deo) si...,Le droit à une telle aide dépend de la législa...,0.248854,0.248854,0.0,0.9,0.000597
3,3,Peut-on saisir ou confisquer la voiture de mes...,Le trafic de drogue peut entraîner des conséqu...,0.194504,0.194504,1.0,1.0,0.000111
4,4,Quels types de décisions relèvent de l'autorit...,Les décisions sur les dépenses liées aux enfan...,0.088229,0.088229,0.0,1.0,0.000625
...,...,...,...,...,...,...,...,...
215,215,Je pense que mon logement est insalubre. Je m'...,"Rassemblez des preuves de l'insalubrité, telle...",0.288132,0.288132,1.0,0.8,0.000173
216,216,Je suis convoqué en justice. Comment vais-je r...,La décision de justice sera délivrée par le ju...,0.096007,0.096007,1.0,0.7,0.000074
217,217,Je divorce. Je divorce par consentement mutuel...,Le divorce par consentement mutuel est un proc...,0.516586,0.516586,1.0,0.9,0.000563
218,218,Je divorce. Je divorce pour cause de désunion ...,"L'abus physique, émotionnel ou psychologique. ...",0.018821,0.018821,0.0,1.0,0.000516


In [110]:
submission = pd.DataFrame({
    "Id": [0],  # Single identifier for the row
    "relevance": [test_df_with_metrics["relevance"].mean()],
    "completeness": [test_df_with_metrics["completeness"].mean()],
    "legal_soundness": [test_df_with_metrics["legal_soundness"].mean()],
    "fluency": [test_df_with_metrics["fluency"].mean()],
    "latency": [test_df_with_metrics["latency"].mean()],
})

In [111]:
# Save the submission DataFrame to a CSV file
submission.to_csv("submission3.csv", index=False)


In [ ]:
# Calculate the composite score for test_df
composite_score = calculate_composite_score(test_df_with_metrics)
print("\nComposite Score for test_df:")
print(composite_score)